# Library Book Recommendation System

---
## Business Understanding
In the increasingly competitive landscape of online libraries and book platforms, personalized recommendation systems have emerged as a critical tool for enhancing user engagement, satisfaction, and retention. Understanding users' unique preferences, demographics, and reading habits is essential for delivering relevant and tailored book suggestions that resonate with individual tastes. By leveraging advanced algorithms and user data analytics, our platform aims to revolutionize the book discovery experience, offering users a curated selection of books that align with their interests and preferences. Through continuous iteration and optimization, we strive to create a dynamic and immersive user interface that not only facilitates book exploration but also fosters a sense of community and interaction among readers, ultimately driving business growth and differentiation in the competitive market.


---
## Problem Statement

The current landscape of online libraries and book platforms lacks personalized recommendation systems that effectively cater to individual users' preferences and reading habits. Existing platforms often rely on generic recommendations or fail to leverage user data to deliver relevant book suggestions, leading to suboptimal user experiences and limited engagement. Our platform aims to address this challenge by developing a robust and intuitive recommendation system that harnesses user data, ratings, and book information to provide personalized book recommendations tailored to each user's unique profile. By seamlessly integrating user registration, profile creation, and recommendation display functionalities into the user interface, we seek to empower users to discover new books that align with their interests and preferences while fostering a vibrant and engaging community of readers.

### Project Objectives
$i.$ Develop a personalized book recommendation engine that utilizes user data, ratings, and book information to generate tailored book suggestions for individual users based on their preferences, demographics, and reading history.

$ii.$ **Create User-Friendly Interface:** Develop an intuitive and user-friendly interface that facilitates seamless user registration, profile creation, and recommendation display. 

### Metrics Of Success

The success of our recommendation engine lies in delivering personalized content to every book lover on our platform, as reflected by high conversion rates of the content provided to users. While the industry standard for mean absolute error (MAE) is 0.5, we aim to surpass these expectations by developing a recommendation engine with a maximum MAE of 0.3. This commitment ensures that our engine consistently provides accurate and tailored book suggestions, exceeding industry benchmarks and enhancing user satisfaction and engagement.



---
## Data Understanding

The used in this project was optained from [Kaggle](https://www.kaggle.com/datasets/syedjaferk/book-crossing-dataset?select=BX-Books.csv). We obtained additional data from [Google Books API](https://developers.google.com/books).

In [1]:
#Importing 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Loading Data**

In [49]:
#From Kaggle
#importing Ratings datasets
ratings = pd.read_csv('data/BX-Book-Ratings.csv', encoding= 'latin-1',sep= ';')

# importing users dataset
users = pd.read_csv('data/BX-Users.csv', encoding= 'latin-1', sep= ';')

# importing books datasets
books = pd.read_csv('data/BX-Books.csv', encoding='latin-1', sep=';', on_bad_lines='skip')


C:\Users\KennedyOdhiambo\AppData\Local\Temp\ipykernel_14120\543443058.py:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('data/BX-Books.csv', encoding='latin-1', sep=';', on_bad_lines='skip')


In [52]:
#checking the records causing the mixed dtypes in year of publication column
books[books['Year-Of-Publication'].str.contains('DK Publishing Inc', na=False)]


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,NaN
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,NaN


In [53]:
# Filtering the rows containing the specified word in the 'Year-Of-Publication' column
books_to_drop = books[books['Year-Of-Publication'].str.contains('DK Publishing Inc', na=False)]

# Droppping the filtered rows from the DataFrame
books = books.drop(books_to_drop.index)

# Resetting the index of the DataFrame after dropping rows
books = books.reset_index(drop=True)


In [54]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
